In [5]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
# Data directory
DATA_DIR = Path('../data/raw/mind-news-dataset/MINDsmall_train')

# Load news.tsv into a DataFrame
news_headers = (
    'news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 
    'title_entities', 'abstract_entities'
)
news_df = pd.read_csv(
    DATA_DIR / 'news.tsv', sep='\t', names=news_headers)

# Load behaviours.tsv into a DataFrame
behaviours_headers = (
    'beh_id', 'user_id', 'timestamp', 'history', 'impressions'
)
behaviours_df = pd.read_csv(
    DATA_DIR / 'behaviors.tsv', sep='\t', names=behaviours_headers)

TypeError: read_csv() got an unexpected keyword argument 'headers'

In [9]:
news_df

,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By","Shop the notebooks, jackets, and more that the royals can't live without.",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"", ""Type"": ""P"", ""WikidataId"": ""Q80976"", ""Confidence"": 1.0, ""OccurrenceOffsets"": [48], ""SurfaceForms"": [""Prince Philip""]}, {""Label"": ""Charles, Prince of Wales"", ""Type"": ""P"", ""WikidataId"": ""Q43274"", ""Confidence"": 1.0, ""OccurrenceOffsets"": [28], ""SurfaceForms"": [""Prince Charles""]}, {""Label"": ""Elizabeth II"", ""Type"": ""P"", ""WikidataId"": ""Q9682"", ""Confidence"": 0.97, ""OccurrenceOffsets"": [11], ""SurfaceForms"": [""Queen Elizabeth""]}]",[]
0,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
1,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
3,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N2073,sports,football_nfl,Should NFL be able to fine players for critici...,Several fines came down against NFL players fo...,https://assets.msn.com/labs/mind/AAJ4lap.html,"[{""Label"": ""National Football League"", ""Type"":...","[{""Label"": ""National Football League"", ""Type"":..."
...,...,...,...,...,...,...,...,...
51276,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...,https://assets.msn.com/labs/mind/BBWzQJK.html,"[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid...","[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid..."
51277,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN,https://assets.msn.com/labs/mind/BBWzQYV.html,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...",[]
51278,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
51279,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]


In [8]:
# Category Distribution
category_count = news_df['Category'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=category_count.index, y=category_count.values)
plt.title('Category Distribution of News Articles')
plt.xlabel('Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

KeyError: 'Category'

In [ ]:

# Chart 2: Subcategory Distribution (Bar Chart)
subcategory_count = news_df['SubCategory'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=subcategory_count.index, y=subcategory_count.values)
plt.title('Subcategory Distribution of News Articles')
plt.xlabel('Subcategory')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()



# Chart 3: Click Through Rate (CTR) Analysis (Histogram)
behaviors_df['CTR'] = behaviors_df['Impressions'].sum(axis=1) / behaviors_df['Impressions'].shape[1]
plt.figure(figsize=(8, 6))
sns.histplot(data=behaviors_df, x='CTR', bins=20, kde=True)
plt.title('Distribution of Click Through Rates (CTR) for News Articles')
plt.xlabel('CTR')
plt.ylabel('Count')
plt.show()
